tft_data.py

mergedata.py, 

metaGen_ordenc.py, 

metaList_ordenc.txt
metaList4.txt

Data_processing_Fixed.py 

metaGenerator.py, (data/) 

위 파일이 데이터 프로세싱 관련 파일입니다. 
// 


encoder.py,

model_catboost.py

dataloader.py

model_woo2.py

----------------


샘플데이터(match_data1.pickle)

## tft api data requesting

In [ ]:
from time import sleep
import requests
import json
from pandas import json_normalize
import pandas as pd
from pandas import DataFrame
import numpy as np

import pickle
from datetime import date
from requests.adapters import Retry, HTTPAdapter

key = "RGAPI-ff3f9cd8-0265-442b-a841-003126f2791f" # 24시간마다 만료되므로, 재 갱신한 key를 받는 작업이 필요합니다. 
today = date.today().isoformat()


def league_summoner(key, tier, country="kr"):
    request = requests.get(
        f"https://{country}.api.riotgames.com/tft/league/v1/{tier}?api_key={key}"
    )
    return json.loads(request.content)


def summoner_info(summonerId, key, n, country="kr"):  # by_name
    try:
        request = requests.get(
            f"https://{country}.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}?api_key={key}",
            timeout=5,
        )
    except requests.Timeout:
        for i in range(5):
            try:
                print(f"Timeout. {i+1}-th Retry...")
                request = requests.get(
                    f"https://{country}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                    timeout=5,
                )
                break
            except requests.Timeout:
                pass

    if request.status_code == 429:
        print("Rate limit exceeded!")
        sleep(120)
        try:
            request = requests.get(
                f"https://{country}.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}?api_key={key}",
                timeout=5,
            )
        except requests.Timeout:
            for i in range(5):
                try:
                    print(f"Timeout. {i+1}-th Retry...")
                    request = requests.get(
                        f"https://{country}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                        timeout=5,
                    )
                    break
                except requests.Timeout:
                    pass

    return json.loads(request.content)


def summoner_info_retry(summonerId, key, country="kr"):  # by_name
    s = requests.Session()
    retries = Retry(total=5, backoff_factor=0, status_forcelist=[502, 503, 504])
    s.mount("https://", HTTPAdapter(max_retries=retries))
    request = s.get(
        f"https://{country}.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}?api_key={key}"
    )
    return json.loads(request.content)


def load_summonerId():
    with open(f"summoner_2022-11-13.pickle", "rb") as f:
        summonerId = pickle.load(f)
    return summonerId


def summoner_puuid(summonerId):
    summ_puuid = pd.DataFrame()
    step = 0
    row_idx = 0
    for id in summonerId:
        step += 1
        si = summoner_info(id, key, "kr")
        si = json_normalize(si)
        try:
            summ_puuid = pd.concat([summ_puuid, si["puuid"]], ignore_index=True)
        except KeyError as e:
            print(id)
            continue
        row_idx += 1
        print(f"step {step} done! : puuid = " + si["puuid"])

    return summ_puuid


def get_puuid(summonerId):
    for i in range(0, 15):
        print(i)
        puuid = summoner_puuid(summonerId[i])
        print(f"{i}-th iter done!")
        with open(f"C:/Users/edwar/TFTML/data/puuid_{i}.pickle", "wb") as f:
            pickle.dump(puuid, f)


def get_matchid(puuid, key, n, region="asia"):
    matchid = []
    step = 0
    for i in puuid:
        step += 1
        try:
            request = requests.get(
                f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                timeout=5,
            )
        except requests.Timeout:
            for i in range(5):
                try:
                    print(f"Timeout. {i+1}-th Retry...")
                    request = requests.get(
                        f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                        timeout=5,
                    )
                    break
                except requests.Timeout:
                    pass

        if request.status_code == 429:
            print("Rate limit exceeded!")
            sleep(120)
            try:
                request = requests.get(
                    f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                    timeout=5,
                )
            except requests.Timeout:
                request = requests.get(
                    f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                    timeout=5,
                )
        request = json.loads(request.content)
        print(request)
        matchid.extend(request)
        print(f"step {step} done!")
    return list(set(matchid))


def match_info(matchid, key, region="asia"):
    game_record = pd.DataFrame()
    step = 0
    for i in matchid:
        step += 1
        try:
            request = requests.get(
                f"https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}",
                timeout=5,
            )
        except requests.Timeout:
            for i in range(5):
                try:
                    print(f"Timeout. {i+1}-th Retry...")
                    request = requests.get(
                        f"https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}",
                        timeout=5,
                    )
                    break
                except requests.Timeout:
                    pass

        if request.status_code == 429:
            print("Rate limit exceeded!")
            sleep(120)
            try:
                request = requests.get(
                    f"https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}",
                    timeout=5,
                )
            except requests.Timeout:
                for i in range(5):
                    try:
                        print(f"Timeout. {i+1}-th Retry...")
                        request = requests.get(
                            f"https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}",
                            timeout=5,
                        )
                    except requests.Timeout:
                        pass
        request = json.loads(request.content)
        request = json_normalize(request)
        print(request)
        game_record = pd.concat([game_record, request])
        print(f"step {step} done!")
    return game_record


""" # summonerId -> puuid -> matchId 받아오는 과정. 이미 받아왔으므로 주석처리했습니다. 학습에선 속도의 용이를 위해 pickle파일로 저장하여 사용했습니다.
summonerId = load_summonerId()
print(summonerId)

ch_matchid_list = []
for i in range(0, 15):
    with open(f"data/puuid_{i}.pickle", "rb") as f:
        puuid = pickle.load(f)
    puuid_list = puuid.values.tolist()
    puuid_list = [i[0] for i in puuid_list]
    ch_matchid = get_matchid(puuid_list, key, 30)
    print(f"{i}-th iter done!")
    with open(f"data/matchid_{i}.pickle", "wb") as f:
        pickle.dump(ch_matchid, f)
"""

for i in range(22, 31):
    with open(
        f"data/merged_match_id/match_id{i}.pickle", "rb"
    ) as f:
        matchid = pickle.load(f)
    match_data = match_info(matchid, key, "asia")
    with open(f"data/match_data/match_data{i}.pickle", "wb") as f:
        pickle.dump(match_data, f)

## Merge data

In [ ]:
from time import sleep
import requests
import json
from pandas import json_normalize
import pandas as pd
from pandas import DataFrame

import pickle

data = pd.DataFrame()
# 1~7
for i in range(1, 8):
    with open(f"data/match_data/match_data{i}.pickle", "rb") as f:
        data_t = pickle.load(f)
    data = pd.concat([data, data_t])

# 9~15
for i in range(9, 16):
    with open(f"data/match_data/match_data{i}.pickle", "rb") as f:
        data_t = pickle.load(f)
    data = pd.concat([data, data_t])

# 22~30
for i in range(22, 31):
    with open(f"data/match_data/match_data{i}.pickle", "rb") as f:
        data_t = pickle.load(f)
    data = pd.concat([data, data_t])

# with open("data/match_data/match_info20.pickle", "rb") as f:
#     data_t = pickle.load(f)

data = pd.concat([data, data_t])

with open(f"data/match_data/match_data.pickle", "wb") as f:
    pickle.dump(data, f)

# data.to_csv("data/match_data.csv", encoding="utf-8")

print(len(data))


## Data Processing 

In [ ]:
import pandas as pd
import csv
import json
import pickle

f = open("data/check.csv", "r")

# f = open("data/match_data/temp_ch_2022-11-10.csv", "r")
# read data from pickle file
df = pd.read_pickle("data/match_data/match_data22.pickle")

MetaData_item = [
    "TFT_Item_ArchangelsStaff",
    "TFT_Item_Bloodthirster",
    "TFT_Item_BrambleVest",
    "TFT_Item_Chalice",
    "TFT_Item_Deathblade",
    "TFT_Item_DragonsClaw",
    "TFT_Item_ForceOfNature",
    "TFT_Item_FrozenHeart",
    "TFT_Item_GargoyleStoneplate",
    "TFT_Item_GuardianAngel",
    "TFT_Item_GuinsoosRageblade",
    "TFT_Item_HextechGunblade",
    "TFT_Item_InfinityEdge",
    "TFT_Item_IonicSpark",
    "TFT_Item_JeweledGauntlet",
    "TFT_Item_LastWhisper",
    "TFT_Item_LocketOfTheIronSolari",
    "TFT_Item_MadredsBloodrazor",
    "TFT_Item_Morellonomicon",
    "TFT_Item_PowerGauntlet",
    "TFT_Item_Quicksilver",
    "TFT_Item_RabadonsDeathcap",
    "TFT_Item_RapidFireCannon",
    "TFT_Item_RedBuff",
    "TFT_Item_Redemption",
    "TFT_Item_RunaansHurricane",
    "TFT_Item_SeraphsEmbrace",
    "TFT_Item_Shroud",
    "TFT_Item_SpearOfShojin",
    "TFT_Item_StatikkShiv",
    "TFT_Item_ThiefsGloves",
    "TFT_Item_TitanicHydra",
    "TFT_Item_TitansResolve",
    "TFT_Item_UnstableConcoction",
    "TFT_Item_WarmogsArmor",
    "TFT_Item_ZekesHerald",
    "TFT_Item_Zephyr",
]

X_ch_add = []
Y_ch = []

Combination_Item = [
    "TFT_Item_BFSword",
    "TFT_Item_ChainVest",
    "TFT_Item_GiantsBelt",
    "TFT_Item_NeedlesslyLargeRod",
    "TFT_Item_NegatronCloak",
    "TFT_Item_RecurveBow",
    "TFT_Item_SparringGloves",
    "TFT_Item_Spatula",
    "TFT_Item_TearOfTheGoddess",
]

# delete 404 Not Found and pairs & turbo mode
df = df[df["status.status_code"].isna()]
df = df[df["info.tft_game_type"] == "standard"]
print(len(df))

for game in df["info.participants"]:
    # game is list of 8 players
    # j has augments, companion, gold_left, last_round, level, placement, players_eliminated, puuid, traits, units
    # j is single player
    for j in game:
        X_ch = []
        traits = []
        items = []
        # trait style is activated number of synergy (e.g. 석호6)
        for trait in j["traits"]:
            traits.append(trait["name"] + str(trait["style"]))
        for champ in j["units"]:
            if champ["itemNames"] != []:
                for item in champ["itemNames"]:
                    if item not in MetaData_item:
                        del item
                if champ["itemNames"][0] == "TFT_Item_ThiefsGloves":
                    items.append(champ["itemNames"][0])
                else:
                    for item in champ["itemNames"]:
                        items.append(item)

        X_ch.extend(j["augments"])  # append -> extend
        X_ch.extend(traits)
        X_ch.extend(items)
        Y_ch.append(j["placement"])
        X_ch_add.append(X_ch)

# print(X_ch_add)
# print(Y_ch)

print(len(X_ch_add))
print(len(X_ch_add[4]))
print(X_ch_add[0])
print("\n\n")


## metaGen_ordenc.py -> metaList_ordenc생성

In [ ]:
# meta generator for ordered hot encoding

import pandas as pd

df = pd.read_pickle("data/match_data/match_data.pickle")

item_set = set()
augment_set = set()
trait_set = set()

Combination_Item = [
    "TFT_Item_B.F.Sword",
    "TFT_Item_BFSword",
    "TFT_Item_ChainVest",
    "TFT_Item_GiantsBelt",
    "TFT_Item_NeedlesslyLargeRod",
    "TFT_Item_NegatronCloak",
    "TFT_Item_RecurveBow",
    "TFT_Item_SparringGloves",
    "TFT_Item_Spatula",
    "TFT_Item_TearOfTheGoddess",
]

df = df[df["status.status_code"].isna()]
df = df[df["info.tft_game_type"] == "standard"]

for game in df["info.participants"]:
    for j in game:
        # trait
        for trait in j["traits"]:
            trait_set.add(trait["name"])

        # item
        for item in j["units"]:
            if item["itemNames"] != []:
                item_list = item["itemNames"]
                if "TFT_Item_ThiefsGloves" in item_list:
                    item_set.add("TFT_Item_ThiefsGloves")
                    break

                for item in item_list:
                    if item not in Combination_Item:
                        item_set.add(item)

        # augment
        for aug in j["augments"]:
            augment_set.add(aug)


print("[item set]\n")
print(item_set)

print("[trait set]\n")
print(trait_set)

print("[augment set]\n")
print(augment_set)

f = open("metaList_ordenc.txt", "w")
for i in sorted(item_set):
    if not i.startswith("TFT_Tutorial"):
        f.write(i + "\n")
for i in sorted(trait_set):
    if not i.startswith("TFT_Tutorial"):
        f.write(i + "\n")
for i in sorted(augment_set):
    if not i.startswith("TFT_Tutorial"):
        f.write(i + "\n")


## meta Generator -> metaList4.txt생성

In [ ]:
import pandas as pd
import json
from pprint import pprint

# f = open("./match_data/temp_ch_2022-11-10.csv", "r")
# f = open("./check.csv", "r")
df = pd.read_pickle("data/match_data/match_data.pickle")

item_set = set()
augment_set = set()
trait_set = set()

Combination_Item = [
    "TFT_Item_B.F.Sword",
    "TFT_Item_BFSword",
    "TFT_Item_ChainVest",
    "TFT_Item_GiantsBelt",
    "TFT_Item_NeedlesslyLargeRod",
    "TFT_Item_NegatronCloak",
    "TFT_Item_RecurveBow",
    "TFT_Item_SparringGloves",
    "TFT_Item_Spatula",
    "TFT_Item_TearOfTheGoddess",
]

df = df[df["status.status_code"].isna()]
df = df[df["info.tft_game_type"] == "standard"]

for game in df["info.participants"]:
    for j in game:
        # trait
        for trait in j["traits"]:
            trait_set.add(trait["name"] + str(trait["tier_current"]))

        # item
        for item in j["units"]:
            if item["itemNames"] != []:
                item_list = item["itemNames"]
                if "TFT_Item_ThiefsGloves" in item_list:
                    item_set.add("TFT_Item_ThiefsGloves")
                    break

                for item in item_list:
                    if item not in Combination_Item:
                        item_set.add(item)

        # augment
        for aug in j["augments"]:
            augment_set.add(aug)

"""
print("[item set]\n")
pprint(item_set)

print("[trait set]\n")
pprint(trait_set)

print("[augment set]\n")
pprint(augment_set)
"""
f = open("metaList4.txt", "w")
for i in sorted(item_set):
    if not i.startswith("TFT_Tutorial"):
        f.write(i + "\n")
for i in sorted(trait_set):
    if not i.startswith("TFT_Tutorial"):
        f.write(i + "\n")
for i in sorted(augment_set):
    f.write(i + "\n")

"""
'TFTTutorial_Demon0',
 'TFTTutorial_Gunslinger0',
 'TFTTutorial_Imperial0',
 'TFTTutorial_Pirate0',
 'TFTTutorial_Ranger0',
 'TFTTutorial_Shapeshifter0',
 'TFTTutorial_Sorcerer0',
 'TFTTutorial_TutorialPrimary1',
 'TFTTutorial_TutorialSecondary1' 
"""

"""
 'TFT4_Item_OrnnAnimaVisage',
 'TFT4_Item_OrnnDeathsDefiance',
 'TFT4_Item_OrnnEternalWinter',
 'TFT4_Item_OrnnInfinityForce',
 'TFT4_Item_OrnnMuramana',
 'TFT4_Item_OrnnObsidianCleaver',
 'TFT4_Item_OrnnRanduinsSanctum',
 'TFT4_Item_OrnnRocketPropelledFist',
 'TFT4_Item_OrnnTheCollector',
 'TFT4_Item_OrnnZhonyasParadox',
 'TFT5_Item_ArchangelsStaffRadiant',
 'TFT5_Item_BloodthirsterRadiant',
 'TFT5_Item_BlueBuffRadiant',
 'TFT5_Item_BrambleVestRadiant',
 'TFT5_Item_ChaliceOfPowerRadiant',
 'TFT5_Item_DeathbladeRadiant',
 'TFT5_Item_DragonsClawRadiant',
 'TFT5_Item_FrozenHeartRadiant',
 'TFT5_Item_GargoyleStoneplateRadiant',
 'TFT5_Item_GiantSlayerRadiant',
 'TFT5_Item_GuardianAngelRadiant',
 'TFT5_Item_GuinsoosRagebladeRadiant',
 'TFT5_Item_HandOfJusticeRadiant',
 'TFT5_Item_HextechGunbladeRadiant',
 'TFT5_Item_InfinityEdgeRadiant',
 'TFT5_Item_IonicSparkRadiant',
 'TFT5_Item_JeweledGauntletRadiant',
 'TFT5_Item_LastWhisperRadiant',
 'TFT5_Item_LocketOfTheIronSolariRadiant',
 'TFT5_Item_MorellonomiconRadiant',
 'TFT5_Item_QuicksilverRadiant',
 'TFT5_Item_RabadonsDeathcapRadiant',
 'TFT5_Item_RapidFirecannonRadiant',
 'TFT5_Item_RedemptionRadiant',
 'TFT5_Item_RunaansHurricaneRadiant',
 'TFT5_Item_ShroudOfStillnessRadiant',
 'TFT5_Item_SpearOfShojinRadiant',
 'TFT5_Item_StatikkShivRadiant',
 'TFT5_Item_SunfireCapeRadiant',
 'TFT5_Item_ThiefsGlovesRadiant',
 'TFT5_Item_TitansResolveRadiant',
 'TFT5_Item_TrapClawRadiant',
 'TFT5_Item_WarmogsArmorRadiant',
 'TFT5_Item_ZekesHeraldRadiant',
 'TFT5_Item_ZephyrRadiant',
 'TFT5_Item_ZzRotPortalRadiant',
 'TFT7_Item_AssassinEmblemItem',
 'TFT7_Item_BruiserEmblemItem',
 'TFT7_Item_CannoneerEmblemItem',
 'TFT7_Item_CavalierEmblemItem',
 'TFT7_Item_DarkflightEmblemItem',
 'TFT7_Item_DarkflightEssence',
 'TFT7_Item_DragonmancerEmblemItem',
 'TFT7_Item_EvokerEmblemItem',
 'TFT7_Item_GuardianEmblemItem',
 'TFT7_Item_GuildEmblemItem',
 'TFT7_Item_JadeEmblemItem',
 'TFT7_Item_LagoonEmblemItem',
 'TFT7_Item_MageEmblemItem',
 'TFT7_Item_MirageEmblemItem',
 'TFT7_Item_MysticEmblemItem',
 'TFT7_Item_ScalescornEmblemItem',
 'TFT7_Item_ShimmerscaleCrownOfChampions',
 'TFT7_Item_ShimmerscaleDeterminedInvestor',
 'TFT7_Item_ShimmerscaleDeterminedInvestor_HR',
 'TFT7_Item_ShimmerscaleDiamondHands',
 'TFT7_Item_ShimmerscaleDiamondHands_HR',
 'TFT7_Item_ShimmerscaleDravensAxe',
 'TFT7_Item_ShimmerscaleDravensAxe_HR',
 'TFT7_Item_ShimmerscaleEmblemItem',
 'TFT7_Item_ShimmerscaleGamblersBlade',
 'TFT7_Item_ShimmerscaleGamblersBlade_HR',
 'TFT7_Item_ShimmerscaleGoldmancersStaff',
 'TFT7_Item_ShimmerscaleGoldmancersStaff_HR',
 'TFT7_Item_ShimmerscaleHeartOfGold',
 'TFT7_Item_ShimmerscaleMogulsMail',
 'TFT7_Item_ShimmerscaleMogulsMail_HR',
 'TFT7_Item_SwiftshotEmblemItem',
 'TFT7_Item_TempestEmblemItem',
 'TFT7_Item_WarriorEmblemItem'
"""